In [1]:
import os, random
import numpy as np
import pandas as pd
import PIL
import keras
import itertools
from PIL import Image
import ibm_boto3
from botocore.client import Config


from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


Using TensorFlow backend.
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_e857a804ea664403af9fdcb1f7be9ae1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WPWL1V9CMU9sL14cHL3hkraySE8fF93PmA-uCFW7v-DX',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_e857a804ea664403af9fdcb1f7be9ae1.get_object(Bucket='cnn-donotdelete-pr-9xck8mbas9jav0', Key='Data_Upload.zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 



In [3]:
from io import BytesIO
import zipfile

zip_ref = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
#print(zip_ref.namelist())
paths = zip_ref.namelist()
#print(path)
#print(zip_ref.printdir())
for path in paths:
    print(zip_ref.extract(path))
zip_ref.close()

/home/dsxuser/work/Data_Upload
/home/dsxuser/work/Data_Upload/test
/home/dsxuser/work/Data_Upload/test/Cheque
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_4.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_5.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_6.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_7.jpg
/home/dsxuser/work/Data_Upload/test/Cheque/cheque_8.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_26.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_27.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_28.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_29.jpg
/home/dsxuser/work/Data_Upload/test/Driving_License/drivinglicense_30.jpg
/home/dsxuser/work/Data_Upload/test/None
/home/dsxuser/work/Data_Upload/test/None/none_15.jpg
/home/dsxuser/work/Data_Upload/test/None/none_16.jpg
/home/dsxuser/work/Data_Upload/test/None/

/home/dsxuser/work/Data_Upload/Trial_val/None/none_29.jpg
/home/dsxuser/work/Data_Upload/Trial_val/None/none_31.jpg
/home/dsxuser/work/Data_Upload/Trial_val/None/none_32.jpg
/home/dsxuser/work/Data_Upload/Trial_val/None/none_33.jpg
/home/dsxuser/work/Data_Upload/Trial_val/None/none_34.jpg
/home/dsxuser/work/Data_Upload/Trial_val/None/none_35.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_25.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_26.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_27.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_28.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_29.jpeg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_30.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_31.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_32.jpg
/home/dsxuser/work/Data_Upload/Trial_val/Pancard/pancard_33.jpg
/home/dsxuser/work/Data_Up

In [4]:
keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())

In [5]:
train_path = '/home/dsxuser/work/Data_Upload/Trial_train/'
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['Cheque', 'Driving_License','None','Pancard','Passport'], batch_size=20)
type(train_batches)

Found 191 images belonging to 5 classes.


keras.preprocessing.image.DirectoryIterator

In [6]:
val_path = '/home/dsxuser/work/Data_Upload/Trial_val/'
val_batches = ImageDataGenerator().flow_from_directory(val_path, target_size=(224,224), classes=['Cheque', 'Driving_License','None','Pancard','Passport'], batch_size=10)


Found 50 images belonging to 5 classes.


In [7]:
test_path = '/home/dsxuser/work/Data_Upload/test/'
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['Cheque', 'Driving_License','None','Pancard','Passport'], batch_size=25)


Found 25 images belonging to 5 classes.


In [8]:
test_imgs, test_labels = next(test_batches)
#plots(test_imgs, titles=test_labels)
test_labels

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.]], dtype=float32)

In [9]:
# Encoding the Dependent Variable
y_test= [ np.where(r==1)[0][0] for r in test_labels ]
y_test

[4, 1, 1, 2, 1, 3, 0, 2, 0, 0, 1, 2, 2, 2, 4, 4, 3, 3, 1, 3, 4, 0, 4, 0, 3]

In [10]:
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary() # The output layer gives 1000, this model has 1000 classes

553467904/553467096 [==============================] - 14s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_p

In [11]:
type(vgg16_model) #This is a Keras Functional API need to convert to sequential
model = Sequential() #Iterate over the functional layers and add it as a stack
for layer in vgg16_model.layers:
    model.add(layer)

In [12]:
model.layers.pop() # We dont want 1000 classes, so remove the last layer
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
for layer in model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
    layer.trainable = False

In [14]:
model.add(Dense(5, activation='sigmoid')) # Add the last layer
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
model.compile(Adam(lr=.00015), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit_generator(train_batches, steps_per_epoch=20, 
                    validation_data=val_batches, validation_steps=20, epochs=5, verbose=1)

Epoch 1/5
20/20 [==============================] - 661s 33s/step - loss: 1.6010 - acc: 0.4496 - val_loss: 1.5970 - val_acc: 0.5600
Epoch 2/5
20/20 [==============================] - 666s 33s/step - loss: 1.5918 - acc: 0.6506 - val_loss: 1.5897 - val_acc: 0.6800
Epoch 3/5
20/20 [==============================] - 669s 33s/step - loss: 1.5830 - acc: 0.8928 - val_loss: 1.5833 - val_acc: 0.8600
Epoch 4/5
20/20 [==============================] - 664s 33s/step - loss: 1.5744 - acc: 0.9605 - val_loss: 1.5759 - val_acc: 0.8800
Epoch 5/5
20/20 [==============================] - 670s 33s/step - loss: 1.5670 - acc: 0.9800 - val_loss: 1.5709 - val_acc: 0.8800


In [ ]:
#model.save('acc94final.h5') 
#from keras.models import load_model
#model = load_model('acc94final.h5')

In [1]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)
predictions

NameError: name 'model' is not defined

In [1]:
predictions
y_pred=[]
for i in range(len(predictions)):
    y_pred.append(np.argmax(predictions[i]))
print(y_pred)
#plots(test_imgs, titles=y_pred)

ctr=0
for i in range(len(y_pred)):
	if y_pred[i] == y_test[i]:
		ctr=ctr+1
print((ctr/len(y_pred))*100)


NameError: name 'predictions' is not defined